In [1]:
import tweepy #https://github.com/tweepy/tweepy
import csv

DEBUG = False

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    user=api.get_user(screen_name=screen_name)
    
    #retweets=([t.user.screen_name for t in new_tweets[0].retweets()])
    
    #return
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        if DEBUG: print "getting tweets before %s" % (oldest)
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,
                                       count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        if DEBUG: print "...%s tweets downloaded so far" % (len(alltweets))
    
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [{'id_str' : tweet.id_str, 
                  'created_at' : tweet.created_at, 
                  'text' : tweet.text.encode("utf-8"),
                  'retweet_count' : tweet.retweet_count,
                  'screen_name' : screen_name}
                 
                 for tweet in alltweets]
    
    
    return outtweets
    
    #write the csv    
#     with open('%s_tweets.csv' % screen_name, 'wb') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text", "retweet_count", "retweet_user_names"])
#         writer.writerows(outtweets)
    
#     pass

tweets = get_all_tweets("ultra")

In [2]:
len(tweets)

3158

In [3]:
original_tweets = [t for t in tweets if t['text'][:2] != 'RT']

In [4]:
len(original_tweets)

1849

In [5]:
ids = [tw["id_str"] for tw in original_tweets]

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
new_ids=[]
for i in ids:
    new_ids.append(i.encode('utf-8'))
new_ids


['836635459388334080',
 '835714498363293696',
 '835660280717795329',
 '835264786451611648',
 '834979320334409728',
 '834812689687851008',
 '834402647184699392',
 '834181527143596033',
 '834093914030215171',
 '834079087249522688',
 '833152910368317440',
 '832439930592837633',
 '832297205126012928',
 '831126416184004610',
 '830615822225776640',
 '830534538761748480',
 '829661271964450816',
 '829549991958032385',
 '829399202811826180',
 '828075671817687040',
 '828061030962102272',
 '828060295138574336',
 '827687977052753921',
 '827188754062438401',
 '826946340299096068',
 '826859462354952192',
 '826104921049149440',
 '825055183013023744',
 '824407954191421441',
 '824023982450110465',
 '823735663916879872',
 '823628529816498177',
 '823557937205493762',
 '822899633932369920',
 '822125780381630468',
 '822111867372011521',
 '821786301951254530',
 '821561927721746432',
 '821561084238819329',
 '821501288210984960',
 '821402882075402240',
 '821402830590320640',
 '819912959581417472',
 '819754435

In [8]:
import time

In [9]:
len(new_ids)

1849

In [35]:
def id_sn_location(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
        
        with open('ultra300.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [36]:
#id_sn_location(new_ids[0:300])

In [37]:
def id_sn_location1000(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
        
        with open('ultra1000.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [38]:
#id_sn_location1000(new_ids[1001:])

In [39]:
import pandas as pd

In [40]:
ultra=pd.read_csv('ultra300.csv')
ultra301=pd.read_csv('ultra2.csv')
ultra1000=pd.read_csv('ultra1000.csv')

In [41]:
ultra.head()

,id,screen_name,location
0,836635459388334080,Mariajosechoez4,NaN
1,836635459388334080,hollymostdope,"Sorrento, FL"
2,836635459388334080,marsi51,México
3,836635459388334080,Jorge7_7Mejias,NaN
4,836635459388334080,Johnny__Munoz,NaN


In [42]:
ultra301.head()

,id,screen_name,location
0,758760829328224256,marquezsaidso,NaN
1,758760829328224256,hottmamaangel,NaN
2,758760829328224256,demas_da_satria,Jakarta-Indonesia
3,758760829328224256,romanromero_dj,Panamá
4,758760829328224256,MelodyLift,Worldwide


In [43]:
ultra1000.head()

,id,screen_name,location
0,613909021515489280,souIpvnk,basspod
1,613909021515489280,kristinxo_,NaN
2,613909021515489280,uhleeez,some place higher
3,613909021515489280,rosexo_,NaN
4,613909021515489280,Brandon_6894,Neue Jersey


In [91]:
ultra2=[ultra, ultra301, ultra1000]

In [92]:
theultra=pd.concat(ultra2)
theultra

,id,screen_name,location
0,836635459388334080,Mariajosechoez4,NaN
1,836635459388334080,hollymostdope,"Sorrento, FL"
2,836635459388334080,marsi51,México
3,836635459388334080,Jorge7_7Mejias,NaN
4,836635459388334080,Johnny__Munoz,NaN
5,836635459388334080,RDNYINC,ѕιιχ-σннн-ƒσuʀʀ
6,836635459388334080,fanny1DZ,"California, USA"
7,836635459388334080,rtrock162,TOKYO JAPAN
8,836635459388334080,AllSeeingTicket,USA
9,836635459388334080,keisuke9965421,ultra fes


In [94]:
theultra.to_pickle('ultra.pkl')

In [93]:
len(theultra['id'].unique())

1750

In [48]:
test=theultra.groupby(['id'], as_index=False).agg(lambda x: set(x))

In [49]:
test2=test.drop(['location'], axis=1)

In [50]:
test3=test2['screen_name'].str.join(sep='*').str.get_dummies(sep='*')

In [51]:
test4=pd.concat([test2, test3], axis=1)

In [52]:
test4.head()

,id,screen_name,00Enriqueumf,031nikitaW,03Pablito,0430Gottsu,04edm,05344b64708046c,060824karats,06_atk_03,...,zrtha_cindy544,zuka_haythem,zulakmxl,zulfialhinduan,zulyculona,zurizadays,zurnaciorumcek,zwolfluke,zzzsilverman,zzzzooooo0eeey
0,516947632435249152,"{_MarleneV, _dianaax3, nanahaam}",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,516955191661568001,"{Pinu5000, DjRickbeats, DanielJ_Torres_, Lex_J...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,516981434322804736,"{Israel_brehh, TheRealEvan159, HeyItsCheyyyy, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,516988345692745728,"{Pinu5000, TheRealEvan159, huseyinsen1978, der...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,516995001856036865,"{josipgrobenski1, aislamfaisal, kiki94392967, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
test4.sum(axis=0)

id                -1248296244991106138
00Enriqueumf                         9
031nikitaW                           1
03Pablito                            1
0430Gottsu                           1
04edm                                1
05344b64708046c                      1
060824karats                         1
06_atk_03                            1
06ta2                                1
07Lmpv                               1
07marciosantos                       1
0831kojikoji                         2
0929Yongi                            1
098Fernandes                         2
09ZQUAD                              1
09_mOnzter                           1
0Xomx                                1
0_0EX                                1
0___4_____6___1                      3
0brienMaguire                        1
0cc2e2c560b34f2                      1
0ceanicskies                         2
0marSebasthian                       1
0mf3d72r7220k6c                      2
0rianaa_06               

In [54]:
test4.sum(axis=0).sort_values(ascending=False)

98dralog                           314
jimmijourney                       157
nishchaymehta17                    149
ridho020                           135
Spoololife                         130
Johnny__Munoz                      119
Fansdashberlin                     106
deepshah__                          93
foryoungjul23                       92
BhuvaneshBp                         89
Listek4                             81
hollymostdope                       78
FedeeeeOK                           75
HirokoLuv                           73
jpsilva_11                          72
David_Brea18_                       69
thohelm                             69
Hardwell_Sonora                     68
cipotpretty                         67
Lupita3Oscar                        67
anndmartinez                        63
Mayra_Enith                         58
pilgrimsindians                     56
anibr_                              53
RAVESTRONGER                        53
musicguru17              

In [55]:
tweets2 = get_all_tweets("98dralog")
tweets3 = get_all_tweets("jimmijourney")
tweets4 = get_all_tweets("nishchaymehta17")

In [56]:
tweets5 = get_all_tweets("ridho020")
tweets6 = get_all_tweets("Spoololife")
tweets7 = get_all_tweets("Johnny__Munoz")

In [59]:
tweets8 = get_all_tweets("Listek4")
tweets9 = get_all_tweets("deepshah__")
tweets10 = get_all_tweets("foryoungjul23")
tweets11 = get_all_tweets("BhuvaneshBp")

In [60]:
print len(tweets2)
print len(tweets3)
print len(tweets4)
print len(tweets5)
print len(tweets6)
print len(tweets7)
print len(tweets8) 
print len(tweets9) 
print len(tweets10)
print len(tweets11)

3212
3122
3152
3208
3151
3201
3224
3224
343
3226


In [61]:
ogt1 = [t for t in tweets2 if t['text'][:2] != 'RT'] 
ogt2 = [t for t in tweets3 if t['text'][:2] != 'RT']
ogt3 = [t for t in tweets4 if t['text'][:2] != 'RT']
ogt4 = [t for t in tweets5 if t['text'][:2] != 'RT']
ogt5 = [t for t in tweets6 if t['text'][:2] != 'RT']
ogt6 = [t for t in tweets7 if t['text'][:2] != 'RT']
ogt7 = [t for t in tweets8 if t['text'][:2] != 'RT']
ogt8 = [t for t in tweets9 if t['text'][:2] != 'RT']
ogt9 = [t for t in tweets10 if t['text'][:2] != 'RT']
ogt10 = [t for t in tweets11 if t['text'][:2] != 'RT']

In [63]:
print len(tweets2), len(ogt1) 
print len(tweets3), len(ogt2) 
print len(tweets4), len(ogt3) 
print len(tweets5), len(ogt4) 
print len(tweets6), len(ogt5) 
print len(tweets7), len(ogt6) 
print len(tweets8) , len(ogt7) 
print len(tweets9) , len(ogt8) 
print len(tweets10), len(ogt9) 
print len(tweets11), len(ogt10)

3212 354
3122 2307
3152 983
3208 12
3151 2414
3201 311
3224 561
3224 1453
343 148
3226 4


In [64]:
ultra_tweets=pd.DataFrame(original_tweets)
ultra_tweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-02-28 17:53:32,836635459388334080,42,ultra,have you reserved your hotel room for #Ultra20...
1,2017-02-26 04:53:57,835714498363293696,41,ultra,#UNCUTAsia -- Episode 5 with @NicoleMoudaber\n...
2,2017-02-26 01:18:31,835660280717795329,743,ultra,Thank you so much Ultra South Africa!\n\nSee y...
3,2017-02-24 23:06:58,835264786451611648,273,ultra,Thank you Ultra South Africa for an incredible...
4,2017-02-24 04:12:37,834979320334409728,94,ultra,"This Saturday, watch the 5th episode of #UNCUT..."


In [65]:
import re

In [66]:
ultra_words=[]
for i in ultra_tweets['text']:
    ultra_words.append(re.sub(r"http\S+","", i))
ultra_words

['have you reserved your hotel room for #Ultra2017 yet?\n\ntime is running out and space is very limited...\n\nBook now: ',
 '#UNCUTAsia -- Episode 5 with @NicoleMoudaber\n\n',
 'Thank you so much Ultra South Africa!\n\nSee you next year!! ',
 'Thank you Ultra South Africa for an incredible first day!! See you tomorrow! ',
 'This Saturday, watch the 5th episode of #UNCUTAsia featuring @NicoleMoudaber! ',
 'The official #Ultra2017 mobile app is out NOW!\n\nAvailable for download at  ',
 'Just TWO DAYS left until Ultra South Africa begins!! ',
 'For #Ultra2017, the VIP Ticket Experience has been upgraded!\n\nFor more information visit  ',
 '@Juandre217 try again, this issue should now be resolved.',
 '@Juandre217 yes! Tickets are available at ',
 '#UNCUTAsia -- Episode 4 featuring the Ultra Angels!\n\n',
 'Become acquainted with the Ultra Angels this Saturday on #UNCUTAsia! ',
 'Ultra Worldwide is now an official @AppleMusic curator!\n\nExplore our dynamic playlists today!\xe2\x80\xa6 ',

In [67]:
retweeters1=pd.DataFrame(tweets2)
retweeters2=pd.DataFrame(tweets3)
retweeters3=pd.DataFrame(tweets4)
retweeters4=pd.DataFrame(tweets5)
retweeters5=pd.DataFrame(tweets6)
retweeters6=pd.DataFrame(tweets7)
retweeters7=pd.DataFrame(tweets8)
retweeters8=pd.DataFrame(tweets9)
retweeters9=pd.DataFrame(tweets10)
retweeters10=pd.DataFrame(tweets11)

In [68]:
frames=[retweeters1,retweeters2,
    retweeters3,retweeters4,
    retweeters5,retweeters6,
    retweeters7,retweeters8,
    retweeters9,retweeters10]

In [69]:
all_retweets=pd.concat(frames)
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-02 02:10:30,837122916160532480,0,98dralog,#EDCMX17 #wasteland \n💀Dead does not die agai...
1,2017-02-28 03:00:32,836410730790404097,0,98dralog,❤One Life - One Love❤\n#EDCMX17\nWe're Going F...
2,2017-02-27 09:48:22,836150978797797376,0,98dralog,Se la RIFO!!!! 😍😍😎😎 PAPI MADA FUCKING GORD...
3,2017-02-27 09:42:48,836149576490967040,0,98dralog,El mejor cierre de todos!!!! Major Lazer!!! 😅...
4,2017-02-26 18:05:51,835913785877618688,0,98dralog,Vamos por el segundo día de EDC! 👊


In [70]:
clean_text=[]
for i in all_retweets['text']:
    clean_text.append(re.sub(r"http\S+","", i))

In [71]:
no_RT=[]
for i in clean_text:
    no_RT.append(re.sub(r"RT", "", i))

In [72]:
retweet_text=pd.DataFrame(no_RT, )

In [73]:
all_retweets['text']=retweet_text
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-02 02:10:30,837122916160532480,0,98dralog,#EDCMX17 #wasteland \n💀Dead does not die agai...
1,2017-02-28 03:00:32,836410730790404097,0,98dralog,❤One Life - One Love❤\n#EDCMX17\nWe're Going F...
2,2017-02-27 09:48:22,836150978797797376,0,98dralog,Se la RIFO!!!! 😍😍😎😎 PAPI MADA FUCKING GORD...
3,2017-02-27 09:42:48,836149576490967040,0,98dralog,El mejor cierre de todos!!!! Major Lazer!!! 😅...
4,2017-02-26 18:05:51,835913785877618688,0,98dralog,Vamos por el segundo día de EDC! 👊


In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords

In [85]:
stopset = set(stopwords.words('english'))
stopset.update(stopwords.words('spanish'))
stopset.update(['love', 'like', 'new', 'amp', 'get', 'one', 'see', 'si', 'today', 'tonight',
               'go','want','good', 'back', 'thank', 'us', 'day','people','make',
                'much', 'know',])

In [86]:
retweets_str=','.join(no_RT)

In [87]:
vectorizer=TfidfVectorizer(stop_words=stopset,use_idf=True, ngram_range=(1,3))
X=vectorizer.fit_transform([retweets_str])

In [88]:
lsa=TruncatedSVD(n_components=1, n_iter=100, random_state=42)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=1, n_iter=100,
       random_state=42, tol=0.0)

In [95]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:20]
    print("Ultra %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

Ultra 0:
edm
asot
edmfamily
soundcloud
dj
music
rave
spoololife
radio
drakemoon
spoololife radio
arminvanbuuren
ultra
shouts
shouts edmfamily
remix
listen
live
groove
time
 
